In [1]:
import pandas as pd
import argparse
import logging
import json
import os
import pathlib
import fiona
import numpy as np
import geopandas as gpd
from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon
from geopandas.tools import geocode
from shapely.geometry import Point
import requests
from zipfile import ZipFile
from tqdm import trange, tqdm

In [2]:
AVG_KMPERYEAR_AUTOMOBILI = 12730 #REF
AVG_KMPERYEAR_AUTOMOBILI_HYB = 12730 #REF
AVG_KMPERYEAR_AUTOVEICOLILEGERI = 14329 #REF
AVG_KMPERYEAR_MOTOVEICOLI = 10000 #REF
AVG_KMPERYEAR_MOTOLEGGERE = 5000 #REF

AVG_LPERKM_AUTOMOBILI = 0.093
AVG_LPERKM_AUTOMOBILI_HYB = 0.065
AVG_LPERKM_AUTOVEICOLILEGERI = 0.131
AVG_LPERKM_MOTOVEICOLI = 0.04
AVG_LPERKM_MOTOLEGGERE = 0.018

pathlib.Path()
NOTEBOOK_PATH = pathlib.Path().resolve()
p = NOTEBOOK_PATH.parent
DATA_DIRECTORY = p / "data"
MOBILITY_DIRECTORY = DATA_DIRECTORY /'mobility'

In [3]:
mobility_file = "TI_mobility_stats.csv"

mobility_num = pd.read_csv(MOBILITY_DIRECTORY/mobility_file).set_index("District")
#mobility_num = pd.read_csv(MOBILITY_DIRECTORY/mobility_file)

Automobili = mobility_num["Automobili"]
Automobili_pkm = Automobili*AVG_KMPERYEAR_AUTOMOBILI

Motoveicoli = mobility_num["Motoveicoli"]
Motoveicoli_pkm = Motoveicoli*AVG_KMPERYEAR_MOTOVEICOLI

Motoleggere = mobility_num["Motoveicoli"]
Motoleggere_pkm = Motoveicoli*AVG_KMPERYEAR_MOTOLEGGERE


## AUTOMOBILY

In [4]:
Automobili_pkm

District
Mendrisio       427295180
Lugano         1225899000
Locarno         486540600
Vallemaggia      47177380
Bellinzona      435926120
Riviera          85265540
Blenio           47941180
Leventina        74992430
Name: Automobili, dtype: int64

In [5]:
TI_carburant_file = "TI_anno_carburante.csv"
Automobility_carburanti = pd.read_csv(MOBILITY_DIRECTORY/TI_carburant_file).set_index("Carburant")

In [6]:
Automobility_carburanti_2020 = Automobility_carburanti['2020']
Automobility_sum_2020 = Automobility_carburanti_2020.sum()

In [7]:
print(Automobility_carburanti_2020, "sum =", Automobility_sum_2020)

Carburant
Essence                                     148394
Diesel                                       63454
Essence-electrique: hybride normal            6274
Essence-electrique: hybride rechargeable      1305
Diesel-electrique: hybride normal              610
Diesel-electrique: hybride rechargeable         38
Electrique                                    1689
Hydrogene                                        0
Gaz (monovalent et bivalent)                   558
Autre                                           69
Name: 2020, dtype: int64 sum = 222391


In [8]:
Automobility_carburanti_2020_share = Automobility_carburanti_2020/Automobility_sum_2020

In [9]:
Automobility_carburanti_2020_share.to_csv("test.csv")
Automobility_carburanti_2020_share

Carburant
Essence                                     0.667266
Diesel                                      0.285326
Essence-electrique: hybride normal          0.028212
Essence-electrique: hybride rechargeable    0.005868
Diesel-electrique: hybride normal           0.002743
Diesel-electrique: hybride rechargeable     0.000171
Electrique                                  0.007595
Hydrogene                                   0.000000
Gaz (monovalent et bivalent)                0.002509
Autre                                       0.000310
Name: 2020, dtype: float64

In [10]:
Auto_pkm_carburant = pd.DataFrame()

for dis in range(len(Automobili_pkm)):
    pkm_carb = Automobili_pkm.iloc[dis]*Automobility_carburanti_2020_share
    Auto_pkm_carburant=  pd.concat([Auto_pkm_carburant, pkm_carb], axis=1)

Auto_pkm_carburant = Auto_pkm_carburant.T
Auto_pkm_carburant["District"] = Automobili_pkm.index
Auto_pkm_carburant = Auto_pkm_carburant.set_index("District", drop = True)
Auto_pkm_carburant.to_csv("private_mobility_pkm_per_carb.csv")

In [11]:
Auto_pkm_carburant

,Essence,Diesel,Essence-electrique: hybride normal,Essence-electrique: hybride rechargeable,Diesel-electrique: hybride normal,Diesel-electrique: hybride rechargeable,Electrique,Hydrogene,Gaz (monovalent et bivalent),Autre
District,,,,,,,,,,
Mendrisio,2.851196e+08,1.219186e+08,1.205467e+07,2.507387e+06,1.172035e+06,73012.023148,3.245192e+06,0.0,1.072124e+06,132574.463085
Lugano,8.180010e+08,3.497812e+08,3.458454e+07,7.193628e+06,3.362539e+06,209469.636811,9.310374e+06,0.0,3.075896e+06,380352.761578
Locarno,3.246521e+08,1.388228e+08,1.372608e+07,2.855041e+06,1.334540e+06,83135.301339,3.695145e+06,0.0,1.220776e+06,150956.205062
Vallemaggia,3.147987e+07,1.346095e+07,1.330948e+06,2.768389e+05,1.294036e+05,8061.209491,3.582995e+05,0.0,1.183725e+05,14637.459340
Bellinzona,2.908788e+08,1.243812e+08,1.229816e+07,2.558033e+06,1.195709e+06,74486.793800,3.310742e+06,0.0,1.093780e+06,135252.336111
Riviera,5.689481e+07,2.432850e+07,2.405475e+06,5.003419e+05,2.338763e+05,14569.341925,6.475689e+05,0.0,2.139393e+05,26454.857706
Blenio,3.198953e+07,1.367888e+07,1.352496e+06,2.813209e+05,1.314987e+05,8191.720169,3.641004e+05,0.0,1.202889e+05,14874.439253
Leventina,5.003991e+07,2.139731e+07,2.115654e+06,4.400588e+05,2.056980e+05,12813.973317,5.695474e+05,0.0,1.881631e+05,23267.477866


In [12]:
Auto_pkm_carburant["Benzina_Ltr"] = Auto_pkm_carburant["Essence"]

SyntaxError: invalid syntax (2808026803.py, line 1)